In [1]:
!python --version

Python 3.5.2


In [1]:
import pandas as pd

In [2]:
# subway_files name list
# subway_files = ["june.csv", "july.csv", "august.csv", "sepetember.csv", "october.csv",]

In [3]:
subway =  pd.read_csv('2010.csv', encoding='utf-8',engine='python')
cols=list(subway.columns)
cols[0]='역명'
subway.columns=cols
subway = subway.loc[subway['역명'] == '신촌(240)']

In [4]:
subway.head(5)

,역명,날짜,구분,05~06,06~07,07~08,08~09,09~10,10~11,11~12,...,16~17,17~18,18~19,19~20,20~21,21~22,22~23,23~24,00~01,01~02
98,신촌(240),2010-01-01,승차,850,798,603,652,687,773,976,...,"1,734","1,984","1,802","1,665","1,618","1,968","2,424","1,515",0,0
99,신촌(240),2010-01-01,하차,104,365,384,497,765,753,"1,068",...,"1,948","2,612","2,724","2,049","1,613","1,264","1,105",966,48,0
330,신촌(240),2010-01-02,승차,313,409,543,837,866,"1,115","1,500",...,"3,085","3,654","3,517","2,981","3,426","3,713","4,503","2,852",1,0
331,신촌(240),2010-01-02,하차,102,316,752,"1,496","2,346","1,829","2,331",...,"3,993","5,120","5,169","3,562","2,390","2,067","1,611","1,263",99,0
562,신촌(240),2010-01-03,승차,454,528,422,615,841,"1,046","1,076",...,"2,551","2,913","2,622","2,541","2,971","2,994","3,305","1,536",1,0


files = ["july.csv", "august.csv", "september.csv", "october.csv"]

# filter other files and concat to base dataframe
for file in files:
    temp = pd.read_csv(file, encoding='euc-kr',engine='python')
    cols=list(temp.columns)
    cols[0]='역명'
    temp.columns=cols
    temp = temp.loc[temp['역명'] == '신촌(240)']
    subway = pd.concat([subway, temp])    

In [5]:
files=['2011.csv','2012.csv','2013.csv','2014.csv']

for f in files:
    temp=pd.read_csv(f,encoding='utf-8',engine='python')
    cols=list(temp.columns)
    cols[0]='역명'
    temp.columns=cols
    temp = temp.loc[temp['역명'] == '신촌(240)']
    subway = pd.concat([subway, temp])   

In [6]:
# convert '날짜' column's format string to datetime
subway_datetime = pd.to_datetime(subway['날짜'])

# convert '날짜' column's format
subway['날짜'] = subway_datetime.dt.strftime('%Y-%m-%d')

In [7]:
subway.shape

(3652, 24)

In [8]:
for column in subway.iloc[:, 3:]:
    subway[column] = subway[column].astype(str).str.replace(",", "").astype(int)

In [9]:
subway['23~26'] = subway['23~24'] + subway['00~01'] + subway['01~02']
subway.drop(['23~24', '00~01', '01~02'], axis=1, inplace=True)

### Get after noon subway dataframe

In [10]:
subway=subway[['날짜', '구분', '16~17', '17~18', '18~19',
       '19~20', '20~21', '21~22', '22~23', '23~26']]

In [11]:
pivot_subway=subway.pivot('날짜','구분')

In [12]:
pivot_subway.head()

16~17       17~18       18~19       19~20       20~21       21~22  \
구분            승차    하차    승차    하차    승차    하차    승차    하차    승차    하차    승차   
날짜                                                                             
2010-01-01  1734  1948  1984  2612  1802  2724  1665  2049  1618  1613  1968   
2010-01-02  3085  3993  3654  5120  3517  5169  2981  3562  3426  2390  3713   
2010-01-03  2551  3468  2913  3745  2622  3553  2541  2433  2971  1727  2994   
2010-01-04  4537  2954  5113  4764  5469  6031  4054  5566  4469  3027  4280   
2010-01-05  4340  3680  4836  4498  5424  6342  4096  6250  4414  3377  4662   

                 22~23       23~26        
구분            하차    승차    하차    승차    하차  
날짜                                        
2010-01-01  1264  2424  1105  1515  1014  
2010-01-02  2067  4503  1611  2853  1362  
2010-01-03  1697  3305  1337  1537   844  
2010-01-04  2249  4828  1980  3165  1950  
2010-01-05  2167  5182  2159  3484  2095

In [13]:
timeslot=['16~17', '17~18', '18~19', '19~20', '20~21', '21~22', '22~23']

for t in timeslot:
    pivot_subway[t+'유입인구']=pivot_subway[t]['하차']-pivot_subway[t]['승차']

In [14]:
incoming_df=pivot_subway[['16~17유입인구', '17~18유입인구', '18~19유입인구', '19~20유입인구', '20~21유입인구', '21~22유입인구', '22~23유입인구','23~26']]

In [15]:
# take only on-board of last train
incoming_df.drop('하차',axis=1,level=1,inplace=True)

# drop level
incoming_df.columns=incoming_df.columns.droplevel(1)

C:\Users\Skywalker\Anaconda3\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [17]:
incoming_df.reset_index(inplace=True)

In [34]:
incoming_df.shape

(1826, 9)

## Weather

In [19]:
# 0) load data
INPUT_FILE_NAME = "2010_weather.csv"
weather = pd.read_csv(INPUT_FILE_NAME, encoding='utf-8')

files = ["2011_weather.csv", "2012_weather.csv", "2013_weather.csv", "2014_weather.csv"]

# filter other files and concat to base dataframe
for file in files:
    temp = pd.read_csv(file, encoding='utf-8',engine='python')
    weather = pd.concat([weather, temp])  

In [20]:
# 1) separate '날짜' column to '날짜', '시간대'

# convert '날짜' column's format string to datetime
weather_datetime = pd.to_datetime(weather['날짜'])

# convet '날짜' column's format
weather['날짜'] = weather_datetime.dt.strftime('%Y-%m-%d')


# helper function for make '시간대' column
def make_hour_interval(hour):
    hour_interval = "{0:02d}~{1:02d}"
    
    return hour_interval.format(hour, hour + 1)
    
#     if hour == 22:
#         return hour_interval.format(hour, 26)
#     else:
#         return hour_interval.format(hour, hour + 1)

    
# make '시간대' column    
weather['시간대'] = weather_datetime.dt.hour.apply(lambda hour: make_hour_interval(hour))

In [21]:
# due to encoding problem. Remove mask
cols=list(weather.columns)
cols[0]='위치'
weather.columns=cols

In [22]:
# 2) 강수량 fillna: NaN -> 0

weather['강수량'].fillna(0, inplace=True)
weather = weather.drop('위치',1)

In [23]:
weather.describe()

C:\Users\Skywalker\AppData\Roaming\Python\Python35\site-packages\numpy\lib\function_base.py:4116: RuntimeWarning: Invalid value encountered in percentile
  interpolation=interpolation)


,기온,강수량,풍속,습도
count,43702.000000,43703.000000,43700.000000,43700.000000
mean,12.530786,0.181722,2.702684,60.413611
std,11.338326,1.527376,1.447386,19.967479
min,-17.700000,0.000000,0.000000,9.000000
25%,NaN,0.000000,NaN,NaN
50%,NaN,0.000000,NaN,NaN
75%,NaN,0.000000,NaN,NaN
max,36.200000,71.000000,11.400000,100.000000


### Check for Null

In [24]:
weather[weather['기온'].isnull()]

,날짜,기온,강수량,풍속,습도,시간대
6833,2011-10-12,NaN,0.0,NaN,NaN,18~19


In [25]:
weather.reset_index(inplace=True)
weather.drop('index',1,inplace=True)
weather.drop('풍속',1,inplace=True)

In [26]:
weather[weather['날짜']=='2011-10-12']

,날짜,기온,강수량,습도,시간대
15551,2011-10-12,13.5,0.0,58.0,00~01
15552,2011-10-12,13.0,0.0,59.0,01~02
15553,2011-10-12,12.5,0.0,60.0,02~03
15554,2011-10-12,12.3,0.0,59.0,03~04
15555,2011-10-12,11.7,0.0,62.0,04~05
15556,2011-10-12,11.1,0.0,65.0,05~06
15557,2011-10-12,11.0,0.0,66.0,06~07
15558,2011-10-12,10.8,0.0,67.0,07~08
15559,2011-10-12,12.2,0.0,62.0,08~09
15560,2011-10-12,13.7,0.0,56.0,09~10


In [27]:
# weather.loc[15569]['기온']=16.0
weather.set_value(15569, '기온', 16.0)
weather.set_value(15569, '습도', 45.0)
weather.set_value(15568, '습도', 46.5)
weather.set_value(15567, '습도', 47.5)

,날짜,기온,강수량,습도,시간대
0,2010-01-01,-11.0,0.0,45.0,01~02
1,2010-01-01,-11.1,0.0,46.0,02~03
2,2010-01-01,-12.1,0.0,54.0,03~04
3,2010-01-01,-12.3,0.0,53.0,04~05
4,2010-01-01,-12.4,0.0,54.0,05~06
5,2010-01-01,-12.4,0.0,57.0,06~07
6,2010-01-01,-12.7,0.0,52.0,07~08
7,2010-01-01,-12.6,0.0,54.0,08~09
8,2010-01-01,-11.7,0.0,43.0,09~10
9,2010-01-01,-9.1,0.0,36.0,10~11


In [28]:
weather.describe()

,기온,강수량,습도
count,43703.000000,43703.000000,43703.000000
mean,12.530865,0.181722,60.412644
std,11.338208,1.527376,19.967137
min,-17.700000,0.000000,9.000000
25%,3.200000,0.000000,45.000000
50%,13.700000,0.000000,60.000000
75%,22.600000,0.000000,77.000000
max,36.200000,71.000000,100.000000


In [29]:
daytime=['22~23', '23~24', '00~01']

In [30]:
night_weather=weather[(weather.시간대 =='22~23') | (weather.시간대 =='23~24') | (weather.시간대 =='00~01')]

In [31]:
night_weather.head()

,날짜,기온,강수량,습도,시간대
21,2010-01-01,-4.8,0.0,54.0,22~23
22,2010-01-01,-4.6,0.0,55.0,23~24
23,2010-01-02,-4.5,0.0,58.0,00~01
45,2010-01-02,-5.5,0.0,73.0,22~23
46,2010-01-02,-6.5,0.0,74.0,23~24


In [32]:
nw_mean=night_weather.groupby(night_weather.날짜)['기온','강수량','습도'].mean()
nw_mean.reset_index(inplace=True)

# Merge subway and weather dataframe

In [125]:
result=incoming_df.merge(nw_mean,on='날짜',how='inner')

In [126]:
# convert '날짜' column's format string to datetime
result['날짜'] = pd.to_datetime(result['날짜'])

In [127]:
result['요일'] = result['날짜'].dt.weekday_name

In [129]:
from pytimekr import pytimekr
from datetime import datetime

In [130]:
def check_holiday(date):
    if date in list(pytimekr.holidays(date.year)):
        return 1
    else:
        
        return 0

In [131]:
result['공휴일']=result['날짜'].dt.date.map(lambda x: check_holiday(x))

In [132]:
result=result[['날짜', '16~17유입인구', '17~18유입인구', '18~19유입인구', '19~20유입인구', '20~21유입인구',
       '21~22유입인구', '22~23유입인구',  '기온', '강수량', '습도', '공휴일','요일','23~26']]

In [137]:
result.loc[result.요일=='Saturday','공휴일']=1

In [139]:
result.loc[result.요일=='Sunday','공휴일']=1

In [140]:
result.to_csv('dataset_metro_weather_sunlok.csv', index = False)